# Modelo
<hr>

In [36]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from joblib import load


def ModeloConsenso(smiles):

    # Calcular huella de morgan
    mol = Chem.MolFromSmiles(smiles)
    fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    fingerprint_list = list(fingerprint)
    finger_array = np.array(fingerprint_list).reshape(1, -1)

    # Crear un DataFrame con nombres de columnas ficticios
    columns = [f'{i}' for i in range(finger_array.shape[1])]
    finger_df = pd.DataFrame(finger_array, columns=columns)

    # Importar modelos
    SVM_model = load("../GuardadoDeModelos/modelo_SVM.joblib")
    RF_model = load("../GuardadoDeModelos/modelo_RF.joblib")
    XGB_model = load("../GuardadoDeModelos/modelo_XGBoost.joblib")

    # Calcular predicciones
    y1 = SVM_model.predict(finger_df)
    y1_prob = SVM_model.predict_proba(finger_df)

    y2 = RF_model.predict(finger_df)
    y2_prob = RF_model.predict_proba(finger_df)

    y3 = XGB_model.predict(finger_df)
    if y3 == 0:
        y3 = ["BBB+"]
    else:
        y3 = ["BBB-"]
    y3_prob = XGB_model.predict_proba(finger_df)

    consensus_probability = ((y1_prob[0][0]+y2_prob[0][0]+y3_prob[0][0])/3)
    clase = None

    if (y1 == ["BBB+"]) and (y2 == ["BBB+"]) and (y3 == ["BBB+"]):
        clase = "BBB+"
    else:
        clase = "BBB-"

    return clase, consensus_probability

# EGFR_ESR1
<hr>

In [37]:
import pandas as pd

In [38]:
EGFR_ESR1_MAPK1 = pd.read_csv("../Resultados de la aplicacion/i_EGFR_ESR1_MAPK1.csv")

In [39]:
EGFR_ESR1_MAPK1

,Molecule ChEMBL ID,Smiles,IC50 EGFR,IC50 MAPK1,IC50 SRC
0,CHEMBL388978,CN[C@@H]1C[C@H]2O[C@@](C)([C@@H]1OC)n1c3ccccc3...,78.0,370.0,9.7
1,CHEMBL4062877,Cc1n[nH]c2ccc(-c3cc(N[C@@H](CO)c4ccccc4)cnc3-c...,5.3,574.7,5.7
2,CHEMBL4105329,Cc1n[nH]c2ccc(-c3cc(N[C@@H](CO)c4ccccc4)cnc3-c...,3.1,960.2,3.9
3,CHEMBL4568087,Cn1cc(-c2cnc3c(-c4csc(C(=O)N[C@@H]5CCCC[C@@H]5...,340.0,230.0,3.0


In [40]:
EGFR_ESR1_MAPK1[["BBB", "Probab"]] = EGFR_ESR1_MAPK1["Smiles"].apply(ModeloConsenso).apply(pd.Series)

In [41]:
EGFR_ESR1_MAPK1

,Molecule ChEMBL ID,Smiles,IC50 EGFR,IC50 MAPK1,IC50 SRC,BBB,Probab
0,CHEMBL388978,CN[C@@H]1C[C@H]2O[C@@](C)([C@@H]1OC)n1c3ccccc3...,78.0,370.0,9.7,BBB+,0.671382
1,CHEMBL4062877,Cc1n[nH]c2ccc(-c3cc(N[C@@H](CO)c4ccccc4)cnc3-c...,5.3,574.7,5.7,BBB+,0.665445
2,CHEMBL4105329,Cc1n[nH]c2ccc(-c3cc(N[C@@H](CO)c4ccccc4)cnc3-c...,3.1,960.2,3.9,BBB-,0.535315
3,CHEMBL4568087,Cn1cc(-c2cnc3c(-c4csc(C(=O)N[C@@H]5CCCC[C@@H]5...,340.0,230.0,3.0,BBB-,0.431750


In [42]:
EGFR_ESR1_MAPK1.to_csv("EGFR_ESR1_MAPK1.csv", index=False)

# EGFR_MAPK1
<hr>

In [43]:
EGFR_MAPK1 = pd.read_csv("../Resultados de la aplicacion/i_EGFR_MAPK1.csv")

In [44]:
EGFR_MAPK1[["BBB", "Probab"]] = EGFR_MAPK1["Smiles"].apply(ModeloConsenso).apply(pd.Series)

In [45]:
EGFR_MAPK1

,Molecule ChEMBL ID,Smiles,IC50 EGFR,IC50 MAPK1,BBB,Probab
0,CHEMBL388978,CN[C@@H]1C[C@H]2O[C@@](C)([C@@H]1OC)n1c3ccccc3...,78.0,370.0,BBB+,0.671382
1,CHEMBL4062877,Cc1n[nH]c2ccc(-c3cc(N[C@@H](CO)c4ccccc4)cnc3-c...,5.3,574.7,BBB+,0.665445
2,CHEMBL290106,Oc1c(Cl)cc(Cl)cc1Sc1cc(Cl)cc(Cl)c1O,614.0,385.0,BBB-,0.043428
3,CHEMBL496,Oc1c(Cl)cc(Cl)c(Cl)c1Cc1c(O)c(Cl)cc(Cl)c1Cl,748.6,196.0,BBB-,0.304240
4,CHEMBL4105329,Cc1n[nH]c2ccc(-c3cc(N[C@@H](CO)c4ccccc4)cnc3-c...,3.1,960.2,BBB-,0.535315
5,CHEMBL4568087,Cn1cc(-c2cnc3c(-c4csc(C(=O)N[C@@H]5CCCC[C@@H]5...,340.0,230.0,BBB-,0.431750


In [46]:
EGFR_MAPK1.to_csv("EGFR_MAPK1.csv", index=False)

# EGFR SRC
<hr>

In [47]:
EGFR_SRC = pd.read_csv("../Resultados de la aplicacion/i_EGFR_SRC.csv")

In [48]:
EGFR_SRC[["BBB", "Probab"]] = EGFR_SRC["Smiles"].apply(ModeloConsenso).apply(pd.Series)

In [50]:
EGFR_SRC.to_csv("EGFR_SRC.csv", index = False)

# MAPK1 SRC
<hr>

In [51]:
MAPK1_SRC = pd.read_csv("../Resultados de la aplicacion/i_MAPK1_SRC.csv")

In [52]:
MAPK1_SRC

,Molecule ChEMBL ID,Smiles,IC50 MAPK1,IC50 SRC
0,CHEMBL388978,CN[C@@H]1C[C@H]2O[C@@](C)([C@@H]1OC)n1c3ccccc3...,370.0,9.7
1,CHEMBL4105329,Cc1n[nH]c2ccc(-c3cc(N[C@@H](CO)c4ccccc4)cnc3-c...,960.2,3.9
2,CHEMBL4062877,Cc1n[nH]c2ccc(-c3cc(N[C@@H](CO)c4ccccc4)cnc3-c...,574.7,5.7
3,CHEMBL4568087,Cn1cc(-c2cnc3c(-c4csc(C(=O)N[C@@H]5CCCC[C@@H]5...,230.0,3.0
4,CHEMBL1336,CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)...,109.5,390.0


In [53]:
MAPK1_SRC[["BBB", "Probab"]] = MAPK1_SRC["Smiles"].apply(ModeloConsenso).apply(pd.Series)

In [54]:
MAPK1_SRC

,Molecule ChEMBL ID,Smiles,IC50 MAPK1,IC50 SRC,BBB,Probab
0,CHEMBL388978,CN[C@@H]1C[C@H]2O[C@@](C)([C@@H]1OC)n1c3ccccc3...,370.0,9.7,BBB+,0.671382
1,CHEMBL4105329,Cc1n[nH]c2ccc(-c3cc(N[C@@H](CO)c4ccccc4)cnc3-c...,960.2,3.9,BBB-,0.535315
2,CHEMBL4062877,Cc1n[nH]c2ccc(-c3cc(N[C@@H](CO)c4ccccc4)cnc3-c...,574.7,5.7,BBB+,0.665445
3,CHEMBL4568087,Cn1cc(-c2cnc3c(-c4csc(C(=O)N[C@@H]5CCCC[C@@H]5...,230.0,3.0,BBB-,0.431750
4,CHEMBL1336,CNC(=O)c1cc(Oc2ccc(NC(=O)Nc3ccc(Cl)c(C(F)(F)F)...,109.5,390.0,BBB+,0.921370


In [55]:
MAPK1_SRC.to_csv("MAPK1_SRC.csv", index = False)